In [ ]:
from solr_manager import SolrManager

url = 'http://localhost:8983/solr/'
core = 'articles'
db = 'sqlite:///../data/articles.db'
solr = SolrManager(url, core, db)

In [ ]:
# run if you want to change an existent schema (in case of error, wait for core to load)
solr.reload_core()

In [ ]:
# run if you want to delete all documents in the core (in case of error, wait for core to load)
solr.clear_documents()

In [ ]:
# in case of error, wait for core to load
schema = 'schema.json'
solr.submit_schema(schema)
solr.apply_stopwords('stopwords.txt')
solr.apply_synonyms('synonyms.txt')

In [ ]:
solr.index_articles()

In [ ]:
# TO DO
# final queries

In [10]:
# TO DO
# query evaluation

In [11]:
solr.close()

Solr container with core articles stopped successfully.
